In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
!pip install -q langchain-openai langchain-core requests

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.messages import BaseMessage
import requests
import os

In [ ]:
#tool create
EXCHANGE_RATE_API_KEY = os.getenv('EXCHANGE_RATE_API_KEY')

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency:str, target_currency:str)->float:
    """
    This function fetches the currency conversion factor bewteen a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}'

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value:float, conversion_rate:Annotated[float,InjectedToolArg]) -> float:
    """
    This function converts a given amount from base currency to target currency
    """
    return base_currency_value * conversion_rate

In [ ]:
result = get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})
print(result)

result = convert.invoke({'base_currency_value':34, 'conversion_rate':87.15})
print(result)

In [ ]:
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

In [ ]:
query = HumanMessage("What is the conversion factor between INR and USD and based on that can you convert 100 inr to usd ?")
messages: list[BaseMessage] = [query]

In [ ]:
messages

In [ ]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
    print(tool_call)
    #execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        #print(tool_message1)
        #fetch this conversion rate
        #append tool message to messages list
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        #print(conversion_rate)
        messages.append(tool_message1)
    #execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        #fetch the current args
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [ ]:
messages

In [ ]:
final_result = llm_with_tools.invoke(messages)
print(final_result.content)